In [ ]:
from astropy.io import fits
import numpy as np
import h5py
import glob
from skimage.transform import downscale_local_mean
from PIL import Image
import copy

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def make_img_map(img, mu_lim=32, pixel_size=0.2, rng=[22,36]):
    imgplot = img + np.random.normal(loc=0, scale=(10**(-0.4*mu_lim)*pixel_size*10.)/3., size=img.shape)
    imgflux = copy.copy(imgplot)
    imgplot = -2.5*np.log10(imgplot) + 2.5*np.log10((pixel_size)**2)
    imgplot[~np.isfinite(imgplot)] = rng[1]
    imgplot[imgplot < rng[0]] = rng[0]
    imgplot[imgplot > rng[1]] = rng[1]
    imgplot = NormalizeData(imgplot)
    imgplot = np.uint8(imgplot * 255)
    
    return imgplot, imgflux

In [ ]:
image_dir = './Magneticum/' # Replace with path containing your images
image_files = glob.glob(image_dir+'*.hdf5')
image_i = 0
mu_lim = 31 # limiting SB, 3 sigma 10" X 10"

make_png = False
make_fits = True

In [ ]:
print(image_files)

In [ ]:
with h5py.File(image_files[image_i], 'r') as f:
    image = f['image'][()]
    
imgplot, imgflux = make_img_map(image, mu_lim=mu_lim, rng=[22,mu_lim+3])

In [ ]:
if make_png:
    imgname = image_files[0][-33:-5]+'.png'
    image_plot = Image.fromarray(np.uint8(downscale_local_mean(imgplot,(4,4))))
    image_plot.save(imgname)

In [ ]:
if make_fits:
    # produce fits image with scaling so that -2.5 log10(f) gives AB magnitudes
    fitsname = image_files[0][-33:-5]+'.fits'
    hdu = fits.PrimaryHDU(imgflux)
    hdul = fits.HDUList([hdu])
    hdul.writeto(fitsname)